In [1]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import _LRScheduler
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from functools import cache
import random
np.random.seed(0)
random.seed(0)


In [2]:
a = torch.arange(24).reshape(12, 2)

In [3]:
a

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18, 19],
        [20, 21],
        [22, 23]])

In [5]:
a.view(2, 6, -1)[0]

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11]])